# AlexNet
![](./Picture/alexnet.png)
1. 学习的特征可以超过手工的特征
2. 在深度学习中，通过卷积层进行卷积操作时，输入图像的大小会在每一步卷积后发生变化。这个变化取决于卷积核的大小、步长（stride）、填充（padding）等参数。下面解释每一步卷积操作后图像大小的变化：
3. 维度计算
   1. **第一个卷积层**：
   - 输入图像大小：假设输入图像的大小为 $W_{\text{in}} \times H_{\text{in}}$，其中 $W_{\text{in}}$ 表示图像的宽度，$H_{\text{in}}$ 表示图像的高度。
   - 卷积核大小：假设卷积核的大小为 $K \times K$，这里是 $11 \times 11$。
   - 步长：假设步长为 $S$，这里是 4。
   - 填充：通常在AlexNet等网络中，第一个卷积层会进行填充操作，以保持特征图大小。因此，可以假设填充大小为 $P$。
   - 计算得到输出特征图的大小为：
     $$
     
     W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} + 2P - K}{S} \right\rfloor + 1
     \\
     H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2P - K}{S} \right\rfloor + 1
     
     $$
   2. **第一个池化层**：
   - 池化窗口大小：假设池化窗口的大小为 $F \times F$，这里是 $3 \times 3$。
   - 步长：假设步长为 $S'$，这里是 2。
   - 计算得到输出特征图的大小为：
     $$
     W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} - F}{S'} \right\rfloor + 1
     \\
     H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} - F}{S'} \right\rfloor + 1
     $$
   3. **后续卷积层**：
   - 输入特征图的大小取决于前一层的输出大小。
   - 根据每层的卷积核大小、步长和填充，可以使用类似的公式来计算每一层的输出特征图大小。
4. 输出通道数表示该卷积层使用了多少个卷积核。
   1. 因此，**如果卷积核数量小于输出通道数，是合理的。这意味着每个输出通道是由多个卷积核共同作用在输入上得到的，这种设计称为多通道卷积。**在这种情况下，每个输出通道都会有自己的一组卷积核，用于提取不同的特征。
5. 数据增强和Dropout：为了防止过拟合，AlexNet 引入了数据增强和 Dropout 技术。数据增强可以通过对图像进行旋转、翻转、裁剪等变换，增加训练数据的多样性，提高模型的泛化能力。Dropout 则是在训练过程中随机删除一定比例的神经元，强制网络学习多个互不相同的子网络，从而提高网络的泛化能力。Dropout简单来说就是在前向传播的时候，让某个神经元的激活值以一定的概率p停止工作，这样可以使模型泛化性更强，因为它不会太依赖某些局部的特征。





In [8]:
import time
import torch
from torch import nn, optim
import torchvision

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),#kernel_size, stride
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            #in_channels, out_channels, kernel_size, stride, padding
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img):
            feature = self.conv(img)
            output = self.fc(feature.view(img.shape[0], -1))
            return output


In [9]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def load_data_fashion_mnist(batch_size, resize=None, root='./Datasets/FashionMNIST'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))

    #将PIL图像或numpy.ndarray转换为Tensor，并且对图像进行归一化处理，即将图像的像素值缩放到[0, 1]范围内。
    trans.append(torchvision.transforms.ToTensor())
    #将一系列的图像转换操作组合成一个整体的转换操作
    transform = torchvision.transforms.Compose(trans)

    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_iter, test_iter

batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
net = AlexNet()
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)



training on  cuda
epoch 1, loss 0.6393, train acc 0.757, test acc 0.856, time 71.4 sec
epoch 2, loss 0.3493, train acc 0.871, test acc 0.881, time 71.4 sec
epoch 3, loss 0.2952, train acc 0.891, test acc 0.894, time 68.6 sec
epoch 4, loss 0.2592, train acc 0.904, test acc 0.899, time 68.7 sec
epoch 5, loss 0.2379, train acc 0.913, test acc 0.901, time 68.5 sec
